In [1]:
from flask import Flask, render_template
from flask_pymongo import PyMongo
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import sys, traceback
import datetime as dt
import traceback

In [2]:
# Visit the mars nasa news site
##browser = Browser("chrome", executable_path=r"\Users\dcohen\Documents\UCBE\Mission-to-Mars\apps\chromedriver.exe", headless=True)
#browser = Browser('chrome', **executable_path, headless=False)

executable_path = {'executable_path':r'C:\Users\dcohen\Documents\UCBE\Mission-to-Mars\chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
def scrape_all():
    # Initiate headless driver for deployment
    #executable_path = {'executable_path': 'chromedriver.exe'}
    #browser = Browser("chrome", executable_path=r"\Users\dcohen\Documents\UCBE\Mission-to-Mars\apps\chromedriver.exe", headless=True)
    #browser = Browser('chrome', **executable_path, headless=False)
    news_title, news_paragraph = mars_news(browser)

    # Run all scraping functions and store results in dictionary
    data = {
        "news_title": news_title,
        "news_paragraph": news_paragraph,
        "featured_image": featured_image(browser),
        "facts": mars_facts(),
        "last_modified": dt.datetime.now()
    }
    browser.quit()
    return data

In [4]:
def mars_news(browser):
    # Visit the mars nasa news site
    url = 'https://mars.nasa.gov/news/'
    browser.visit(url)
    
    # Optional delay for loading the page
    browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)
    
    # Convert the browser html to a soup object and then quit the browser
    html = browser.html
    news_soup = BeautifulSoup(html, 'html.parser')
    
    # Add try/except for error handling
    try:
        slide_elem = news_soup.select_one("ul.item_list li.slide")
        #slide_elem.find("div", class_="content_title")
        # Use the parent element to find the first 'a' tag and save it as 'news_title'
        news_title = slide_elem.find("div", class_="content_title").get_text()
        # Use the parent element to find the paragraph text
        # #news_paragraph = slide_elem.find("div", class_="article_teaser_body").get_text()
        news_paragraph = slide_elem.find("div", class_="article_teaser_body").get_text()
    except AttributeError:
        return None, None
    return news_title, news_paragraph

In [5]:
# ""### Featured Images"
# Visit URL
def featured_image(browser):
    url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
    browser.visit(url)

    # Find and click the full image button
    full_image_elem = browser.find_by_id('full_image')
    full_image_elem.click()

    # Find the more info button and click that
    browser.is_element_present_by_text('more info', wait_time=1)
    #browser.click_link_by_partial_text('more info')
    browser.links.find_by_partial_text('more info')
    #more_info_elem = browser.find_link_by_partial_text('more info')
    #more_info_elem.click()

    # Parse the resulting html with soup
    html = browser.html
    img_soup = BeautifulSoup(html, 'html.parser')

    
    try:
        img_url_rel = img_soup.select_one("figure.lede a img").get("src")
    
    except AttributeError:
        return None
    # Use the base URL to create an absolute URL
    img_url = f'https://www.jpl.nasa.gov{img_url_rel}'
    return img_url

In [6]:
def mars_facts():
    # Add try/except for error handling
    try:
        # Use 'read_html' to scrape the facts table into a dataframe
        df = pd.read_html('http://space-facts.com/mars/')[0]
    
    except BaseException:
        return None
    
    # Assign columns and set index of dataframe
    df.columns=['Description', 'Mars']
    df.set_index('Description', inplace=True)
    
    # Convert dataframe into HTML format, add bootstrap
    return df.to_html()

In [7]:
if __name__ == "__main__":
    # If running as script, print scraped data
    print(scrape_all())

{'news_title': "NASA's Perseverance Rover Mission Getting in Shape for Launch", 'news_paragraph': 'Stacking spacecraft components on top of each other is one of the final assembly steps before a mission launches to the Red Planet. ', 'featured_image': None, 'facts': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n   

In [8]:
app = Flask(__name__)

In [ ]:
# Use flask_pymongo to set up mongo connection
app.config["MONGO_URI"] = "mongodb://localhost:27017/mars_app"
mongo = PyMongo(app)

# Visit htlm and look at home page
@app.route("/")
def index():
   mars = mongo.db.mars.find_one()
   return render_template("index.html", mars=mars)

@app.route("/scrape")
def scrape():
   mars = mongo.db.mars
   mars_data = scraping.scrape_all()
   mars.update({}, mars_data, upsert=True)
   return "Scraping Successful!"  

if __name__ == "__main__":
   app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
[2020-05-17 10:51:09,346] ERROR in app: Exception on /scrape [GET]
Traceback (most recent call last):
  File "C:\Users\dcohen\.conda\envs\PythonData\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\dcohen\.conda\envs\PythonData\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\dcohen\.conda\envs\PythonData\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\dcohen\.conda\envs\PythonData\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\dcohen\.conda\envs\PythonData\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\dcohen\.conda\envs\PythonData\lib\site-packages\flask\app.py", line 1935, in dispatch_request
   

127.0.0.1 - - [17/May/2020 12:39:18] "GET / HTTP/1.1" 500 -
127.0.0.1 - - [17/May/2020 12:39:20] "GET /favicon.ico HTTP/1.1" 404 -
[2020-05-17 12:44:40,982] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "C:\Users\dcohen\.conda\envs\PythonData\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\dcohen\.conda\envs\PythonData\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\dcohen\.conda\envs\PythonData\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\dcohen\.conda\envs\PythonData\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\dcohen\.conda\envs\PythonData\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\dcohen\.conda\envs\PythonData\li

127.0.0.1 - - [17/May/2020 16:06:09] "GET / HTTP/1.1" 500 -
[2020-05-17 16:06:20,134] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "C:\Users\dcohen\.conda\envs\PythonData\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\dcohen\.conda\envs\PythonData\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\dcohen\.conda\envs\PythonData\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\dcohen\.conda\envs\PythonData\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\dcohen\.conda\envs\PythonData\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\dcohen\.conda\envs\PythonData\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    retur